<a href="https://colab.research.google.com/github/gabipalacios/Python/blob/main/CZV1D_marco_SinSesgoPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import io
from google.colab import files
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sn
pd.options.display.float_format = '{:.2f}'.format #Setear opciones globales, todo lo que ejecutemos desde pandas, flotantes y decimales, vana a mostrarse 2 decimales.




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Se tomo bbdd de Plataforma, de toda la originación de un período, y se complementa con datos de la cartera de préstamos. Para esto se crea una id compuesta con cédula y mes originación.
origen = pd.read_csv('/content/drive/MyDrive/1Empleos/VALOR/1bbddPhyton/bbdd_sinpagos.csv')

#Se toman algunos parametros del marco BCU de valor, algunos directos y otros son calculados del mismo marco
marco = pd.read_csv('/content/drive/MyDrive/1Empleos/VALOR/1bbddPhyton/bbdd_PreMarcoBCU.csv')

In [ ]:
df = pd.merge(origen, marco, on='Documento', how='inner')


In [ ]:
print(origen.shape)
print(marco.shape)
print(df.shape)

(2700, 73)
(2445, 13)
(1948, 85)


In [ ]:
marco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2445 entries, 0 to 2444
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Documento           2445 non-null   int64  
 1   Entidades           2445 non-null   int64  
 2   Exposición          2445 non-null   int64  
 3   Deuda               2445 non-null   int64  
 4   Rel_DeudaPrincipal  2445 non-null   float64
 5   Contingencia        2445 non-null   int64  
 6   Var_dueda_3M        2445 non-null   float64
 7   Sin_Ficha           2445 non-null   int64  
 8   Malos               2445 non-null   int64  
 9   Mejora              2445 non-null   int64  
 10  Grupo_b             2445 non-null   object 
 11  Grupo_i             2445 non-null   object 
 12  Q_irregular_12m     2445 non-null   int64  
dtypes: float64(2), int64(9), object(2)
memory usage: 248.4+ KB


In [ ]:
marco.describe()

,Documento,Entidades,Exposición,Deuda,Rel_DeudaPrincipal,Contingencia,Var_dueda_3M,Sin_Ficha,Malos,Mejora,Q_irregular_12m
count,2445.00,2445.00,2445.00,2445.00,2445.00,2445.00,2445.00,2445.00,2445.00,2445.00,2445.00
mean,38410740.11,3.65,355559.17,312354.36,0.56,0.63,0.85,0.29,0.16,0.27,0.98
std,10945303.70,2.16,594500.85,559457.87,0.24,0.48,2.03,0.71,0.53,1.12,1.87
min,9284774.00,1.00,0.00,0.00,0.00,0.00,-1.00,0.00,0.00,-5.00,0.00
25%,31716874.00,2.00,71884.00,57138.00,0.38,0.00,-0.01,0.00,0.00,0.00,0.00
50%,40941181.00,3.00,164374.00,139288.00,0.52,1.00,0.13,0.00,0.00,0.00,0.00
75%,47123946.00,5.00,405034.00,343265.00,0.73,1.00,0.71,0.00,0.00,0.00,1.00
max,65521382.00,13.00,9341142.00,9180499.00,1.00,1.00,10.00,3.00,3.00,6.00,12.00


In [ ]:
#Elimino campos que no tengo el dato de plataforma, pero si de Financial, son prestamos que si bien
#se originan en plataforma, se termienan prestamos por terceros. Tengo los datos de Monto, Cuota y tasa.
#cardo orig, seguro y cargo adm los saco porque además de faltar datos, son resultados que no aportan valor al modelo
df = df.drop('monto', axis=1)
df = df.drop('cuota', axis=1)
df = df.drop('tasa', axis=1)
df = df.drop('plazo', axis=1)
df = df.drop('cargo_originacion', axis=1)
df = df.drop('tiene_seguro', axis=1)
df = df.drop('tiene_cargo_adm', axis=1)
df = df.drop('clr_score_if', axis=1)
df = df.drop('clr_bp_fecha', axis=1)
df = df.drop('rpf_alert', axis=1)
df = df.drop('cdr_tiene_quita', axis=1)
df = df.drop('cdr_tiene_oca', axis=1)
df = df.drop('cdr_tiene_oca.1', axis=1)
df = df.drop('cdr_tiene_financiera', axis=1)
df = df.drop('cdr_tiene_brou', axis=1)
df = df.drop('cdr_tiene_banco_privado', axis=1)
df = df.drop('cdr_sum_deudas_ui', axis=1)
df = df.drop('cdr_periodo', axis=1)
df = df.drop('cdr_peor_calificacion', axis=1)
df = df.drop('cdr_max_contingencia_ui', axis=1)
df = df.drop('Tipo_Producto', axis=1)
df = df.drop('Tipo Basico', axis=1)
df = df.drop('clr_fecha', axis=1)
df = df.drop('Canal', axis=1)
df = df.drop('no_of_desc', axis=1)
df = df.drop('id_solicitud', axis=1)
df = df.drop('DesembolsoLugar', axis=1)
df = df.drop('estado_tc', axis=1)
df = df.drop('estado_pa', axis=1)
df = df.drop('estado_cliente', axis=1)
df = df.drop('cargo', axis=1)
df = df.drop('meses_con_pagos', axis=1)
df = df.drop('dias_atraso_max', axis=1)
df = df.drop('DesembolsoSubAge.', axis=1)
df = df.drop('DesembolsoAgencia', axis=1)
df = df.drop('DesembolsoAgencia.1', axis=1)
df = df.drop('CuotasPagas', axis=1)
df = df.drop('localidad', axis=1)
df = df.drop('Id_Cliente', axis=1)
df = df.drop('Atraso', axis=1)
df = df.drop('Solicitud', axis=1)
df = df.drop('Fecha', axis=1)
df = df.drop('Documento', axis=1)
df = df.drop('tiene_tel_fijo', axis=1)
df = df.drop('tiene_recibo_sueldo', axis=1)
df = df.drop('TEA', axis=1)
df = df.drop('plazo_hasta', axis=1)
df = df.drop('monto_maximo', axis=1)
df = df.drop('pricing', axis=1)
df = df.drop('ingreso_estimado', axis=1)


KeyError: "['pricing'] not found in axis"

In [ ]:
df.shape

In [ ]:
#Quitar todos los registros que en las que el campo 'clr_score' no tenga datos, hasta conseguir los faltantes

# Eliminar filas donde 'clr_score' es nulo
df = df.dropna(subset=['clr_score'])

df.info()


In [ ]:
df.value_counts('dispositivo_marca')

In [ ]:
# En el campo [dispositivo_marca], sólo si los valores son distintos de: 'Apple' imputar con valor 'Android' y si no hay datos imputar NaN

# Imputar valores en 'dispositivo_marca'
df['dispositivo_marca'] = np.where(df['dispositivo_marca'].isin(['Apple']), 'Apple',
                                   np.where(df['dispositivo_marca'].notnull(), 'Android','Android'))


In [ ]:
# Imputar 'Other' en 'sist_op_familia' si es distinto de 'Android', 'Other' o 'iOS'
df['sist_op_familia'] = np.where(df['sist_op_familia'] == 'iOS', 'iOS', 'Android')



In [ ]:
df.value_counts('sist_op_familia')

In [ ]:
# prompt: Si existen datos en el campo [dispositivo_marca] vacíos imputar 'Sin dato'

# Imputar valores en 'dispositivo_marca'
#df['dispositivo_marca'] = np.where(df['dispositivo_marca'].isnull(), 'Other', df['dispositivo_marca'])
df['dispositivo_marca'] = np.where(df['dispositivo_marca'] == 'Apple', 'Apple', 'Android')


In [ ]:
df.value_counts('dispositivo_marca')

In [ ]:
# Imputar 'Web' en 'navegador_familia' si es distinto de 'Facebook' o 'Instagram', dado que donde se ejecutan las campañas pueden venir por esos 3 canales.
df['navegador_familia'] = np.where(df['navegador_familia'].isin(['Facebook', 'Instagram']), df['navegador_familia'], 'Web')


In [ ]:
df.value_counts('navegador_familia')

In [ ]:
df['departamento'] = np.where(df['departamento'].isin(['MONTEVIDEO', 'CANELONES']), df['departamento'], 'RESTO')


In [ ]:
df.value_counts('departamento')

In [ ]:
# Revisar valores faltantes

#valores_faltantes = df.isnull().sum()
#valores_faltantes

In [ ]:
df.value_counts('sist_op_familia')

In [ ]:
#Si el valor del campo Zero es Zero imputar 1 y si no 0 (en el dataset original están considerados además los 'malos' cuyo atraso pasa los 60 días)
df['Zero'] = np.where(df['Zero'] == 'Zero', 1, 0)


In [ ]:
# Revisar valores faltantes

valores_faltantes = df.isnull().sum()
#valores_faltantes

In [ ]:
#Modificar el parámetro de la tabla df['hora'] donde se muestran las horas del día desde 0 a 23, son tipos de datos periódicos, utilizando el cálculo trigonométrico más adecuado para introducir el parámetro en un modelo predictivo.

# Convertir la columna 'hora' a valores numéricos entre 0 y 23
df['Hora'] = pd.to_numeric(df['Hora'])

# Calcular la representación sinusoidal de la hora
df['hora_sin'] = np.sin(2 * np.pi * df['Hora'] / 24)
df['hora_cos'] = np.cos(2 * np.pi * df['Hora'] / 24)

# Eliminar la columna original 'hora'
df = df.drop('Hora', axis=1)


In [ ]:
# Imputar 'Web' en 'navegador_familia' si es distinto de 'Facebook' o 'Instagram', dado que donde se ejecutan las campañas pueden venir por esos 3 canales.
df['clr_accion'] = np.where(df['clr_accion'].isin(['C2', 'C1_3_3','C4','C1_2_3','C1_3_6','C1_3_1','C3']), df['clr_accion'], 'Varios')


In [ ]:
df.value_counts('clr_accion')

In [ ]:
df['nivel_educativo'] = df['nivel_educativo'].replace('Educación Básica', 'Primaria')
df['nivel_educativo'] = df['nivel_educativo'].replace('Educación Secundaria', 'Secundaria')

le = LabelEncoder()

df['relacion_laboral'] = le.fit_transform(df['relacion_laboral']) + 1
df['nivel_educativo'] = le.fit_transform(df['nivel_educativo']) + 1
df['sist_op_familia'] = le.fit_transform(df['sist_op_familia']) + 1
df['navegador_familia'] = le.fit_transform(df['navegador_familia']) + 1

df['status_usuario'] = le.fit_transform(df['status_usuario']) + 1
df['status'] = le.fit_transform(df['status']) + 1
df['genero'] = le.fit_transform(df['genero']) + 1
df['dispositivo_marca'] = le.fit_transform(df['dispositivo_marca']) + 1
df['departamento'] = le.fit_transform(df['departamento']) + 1
df['cel_carrier'] = le.fit_transform(df['cel_carrier']) + 1

# Crear el nuevo campo numérico combinando 'nivel_educativo' y 'relacion_laboral'
df['educacion_laboral'] = df['nivel_educativo'] * 10 + df['relacion_laboral']

# Eliminar los campos originales
df = df.drop(['nivel_educativo', 'relacion_laboral'], axis=1)

#df.value_counts('educacion_laboral')

In [ ]:
df['educacion_laboral'] = df['educacion_laboral'].astype('category')
df['nivel_antiguedad_laboral'] = df['nivel_antiguedad_laboral'].astype('category')
df['navegador_familia'] = df['navegador_familia'].astype('category')
df['sist_op_familia'] = df['sist_op_familia'].astype('category')
df['clr_accion'] = df['clr_accion'].astype('category')
df['Día'] = df['Día'].astype('category')
df['Semana_año'] = df['Semana_año'].astype('category')
df['Quincena'] = df['Quincena'].astype('category')
df['Num_Dia_Sem'] = df['Num_Dia_Sem'].astype('category')
df['status_usuario'] = df['status_usuario'].astype('category')
df['status'] = df['status'].astype('category')
df['genero'] = df['genero'].astype('category')
df['dispositivo_marca'] = df['dispositivo_marca'].astype('category')
df['departamento'] = df['departamento'].astype('category')
df['cel_carrier'] = df['cel_carrier'].astype('category')

In [ ]:
categorical_cols = df.select_dtypes(exclude=np.number).columns
for col in categorical_cols:
    print(f"Valores únicos para la columna '{col}':")
    print(df[col].unique())
    print("-" * 20)

In [ ]:
df = df.rename(columns={
    'sist_op_familia': 'sist_op',
    'nivel_antiguedad_laboral': 'antiguedad',
    'navegador_familia': 'navegador',
    'mejor_esquema_pricing': 'pricing',
    'dispositivo_marca': 'marca',
    'Cuotas': 'plazo_elegido',
    'Capital': 'monto_retirado',
    'dispositivo_marca': 'marca',
    'status_usuario': 'status_u',
})

In [ ]:
df['clr_accion'] = le.fit_transform(df['clr_accion']) + 1

In [ ]:
df.info()

-------OOOOOOO-------

In [ ]:
pd.set_option("display.max_columns", None)  # Show all columns
display(df.head(5)) # Display first 5 rows with all columns

In [ ]:

categorical_cols = df.select_dtypes(exclude=np.number).columns
df[categorical_cols].head()


In [ ]:
numeric_cols = df.select_dtypes(include=np.number).columns
df[numeric_cols].head()


In [ ]:
numeric_cols = df.select_dtypes(include=np.number).columns
df[numeric_cols].head()


In [ ]:
df[numeric_cols].describe()

In [ ]:

entidades_quantiles = df['Entidades'].quantile([0.25, 0.5, 0.75])

# Discretize 'Entidades' based on quantiles
df['Entidades_disc'] = pd.cut(df['Entidades'], bins=[-np.inf] + entidades_quantiles.tolist() + [np.inf], labels=False, include_lowest=True)

# Replace the original 'Entidades' column with the discretized values
df['Entidades'] = df['Entidades_disc']
df = df.drop('Entidades_disc', axis=1)


# Calculate quantiles for 'Rel_DeudaPrincipal'
rel_deuda_quantiles = df['Rel_DeudaPrincipal'].quantile([0.25, 0.5, 0.75])

# Discretize 'Rel_DeudaPrincipal' based on quantiles
df['Rel_DeudaPrincipal_disc'] = pd.cut(df['Rel_DeudaPrincipal'], bins=[-np.inf] + rel_deuda_quantiles.tolist() + [np.inf], labels=False, include_lowest=True)

# Replace the original 'Rel_DeudaPrincipal' column with the discretized values
df['Rel_DeudaPrincipal'] = df['Rel_DeudaPrincipal_disc']
df = df.drop('Rel_DeudaPrincipal_disc', axis=1)


In [ ]:
df[numeric_cols].describe()

In [ ]:
# prompt: Preparar las variables de exposición y deuda para aplicar a un modelo predictivo, evaluar la mejor forma de obtener mejores resultados

# Identify features for scaling (numerical features excluding 'Zero')
# Removed 'Grupo_b' and 'Grupo_i' as they might contain string values
features_to_scale = ['Exposición', 'Deuda', 'Rel_DeudaPrincipal', 'Var_dueda_3M', 'Q_irregular_12m', 'monto_retirado', 'plazo_elegido', 'clr_score', 'edad']

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the selected features
df[features_to_scale] = scaler.fit_transform(df[features_to_scale])

# Display the updated DataFrame
print(df.head())


In [ ]:
#Evaluar incoherencias en descripción de variales numéricas, valores dentro de rangos razonables
df.describe()

In [ ]:
# Mostrar campos categóricos
categorical_cols = df.select_dtypes(exclude=np.number).columns
df[categorical_cols].head()


In [ ]:
df.columns

In [ ]:
corrMatt = df[['Día', 'Semana_año', 'Quincena', 'Num_Dia_Sem', 'Zero', 'status_u',
       'status', 'sist_op', 'antiguedad', 'navegador', 'ingreso_declarado',
       'genero', 'edad', 'marca', 'departamento', 'plazo_elegido', 'clr_score',
       'clr_accion', 'cel_carrier', 'monto_retirado', 'Entidades',
       'Exposición', 'Deuda', 'Rel_DeudaPrincipal', 'Contingencia',
       'Var_dueda_3M', 'Sin_Ficha', 'Malos', 'Mejora',
       'Q_irregular_12m', 'hora_sin', 'hora_cos', 'educacion_laboral']].corr() # Removed 'Grupo_b', 'Grupo_i'
mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False
sn.heatmap(corrMatt, mask=mask,
           vmax=.8, square=True,annot=True)

In [ ]:
# ... (previous code) ...
df['clr_accion'] = le.fit_transform(df['clr_accion']) + 1
# ... (previous code) ...

# Convert 'educacion_laboral' to numerical representation using Label Encoding
df['educacion_laboral'] = le.fit_transform(df['educacion_laboral']) + 1

# Convert all categorical columns to numerical using one-hot encoding
categorical_cols = df.select_dtypes(exclude=np.number).columns
# Using pandas get_dummies for one-hot encoding
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Define 'Zero' como la variable objetivo
y = df['Zero']

# Divide los datos en conjuntos de entrenamiento y prueba
X_df = df.drop('Zero', axis=1)

num_data_points = len(X_df)
cut = int(.7 * num_data_points)

X_train = X_df[:cut]
y_train = y[:cut] # Usa la variable objetivo correcta 'y'
X_test = X_df[cut:]
y_test = y[cut:]
print(X_train.shape, X_test.shape)
# ... (rest of the code) ...

In [ ]:
from sklearn import linear_model

logistic = linear_model.LogisticRegression(max_iter=1000, solver='liblinear')
logistic.fit(X_train, y_train)

In [ ]:
#Evalúo el resultado del modelo LogisticRegression.


# Predict on the test set
y_pred = logistic.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=['Predicted 0', 'Predicted 1'],
            yticklabels=['Actual 0', 'Actual 1'])
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:

feature_importances = logistic.coef_[0]

# Create a DataFrame to display feature importances
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})

# Sort by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Display the top N features (e.g., top 10)
print(feature_importance_df.head(10))


# Visualize feature importances
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['Feature'][:10], feature_importance_df['Importance'][:10])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Top 10 Feature Importances (Logistic Regression)')
plt.show()
